# IMPORT

In [ ]:
# import pandas lib as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('mode.chained_assignment', None)

# GET DATA

In [ ]:
# Get Meter List and daily data 
MeterList = pd.read_excel('../Data/19193111_MeterList.xls')
DailyForwardEnergy = pd.read_excel('../Data/19193111_Daily_Forward_Energy.xls')
DailyReverseEnergy = pd.read_excel('../Data/19193111_Daily_Reverse_Energy.xls')
DailyVoltage = pd.read_excel('../Data/19193111_Daily_Voltage.xls')
DailyCurrent = pd.read_excel('../Data/19193111_Daily_Current.xls')


print('- DailyForwardEnergy: print 02 top row ----------------')
print(DailyForwardEnergy.head(2))
print('- DailyReverseEnergy: print 02 top row ----------------')
print(DailyReverseEnergy.head(2))
print('- DailyVoltage: print 02 top row ----------------')
print(DailyVoltage.head(2))
print('- DailyCurrent: print 02 top row ----------------')
print(DailyCurrent.head(2))

# CHECK METER LIST

In [ ]:
# To check data. We must know how many active meter need to collect data
print('- MeterList: print 02 top row ----------------')
print(MeterList.head(2))
# Get Total Meter
TotalMeter = MeterList.shape[0]
print('\n- Total Meter: {total}'.format(total=TotalMeter))

# Get Meter have status is Active
ActiveMeterList = MeterList[(MeterList.Status == 'Running')]
TotalActiveMeter = ActiveMeterList.shape[0]
print('\n- Total Active Meter : {totalActive}'.format(totalActive=TotalActiveMeter))

# Get Meter Model : MM11-V1 is single phase meter, MM31/38 is Three phase meter
MeterModel = ActiveMeterList.Meter_Model.drop_duplicates()
print('\n- Meter Model:\n {first}'.format(first=MeterModel))
SinglePhaseMeter = ActiveMeterList[(ActiveMeterList.Meter_Model == 'MM11-V1')]
TotalSinglePhaseMeter = SinglePhaseMeter.shape[0]
ThreePhaseMeter = ActiveMeterList[(ActiveMeterList.Meter_Model == 'MM31/38')]
TotalThreePhaseMeter = ThreePhaseMeter.shape[0]

print('\n- Total Sing Meter : {first}; Total Three Phase Meter : {second}'.format(first=TotalSinglePhaseMeter,second=TotalThreePhaseMeter))


# CHECK DAILY DATA
With daily data, we need to check the data collect from meter enough or not, check the data is normal or not

DEF function for Check Daily Data

In [ ]:
def getDataRate(ActiveMeterList,DailyData,DataName):
    # Success Rate (how many meter have this data in day per total active meter)
    DailyData['Flag_Time_By_Day'] = DailyData.Data_Time.str.slice(0, 10)

    # Join Voltage data with meter List by Meter_No
    ActiveMeterListWithDailyData=ActiveMeterList.set_index('Meter_No').join(DailyData.set_index('Meter_No'))
    # print('\n- Print Top 02 row of Voltage data with meter List by Meter_No:\n {first}'.format(first=ActiveMeterListWithVoltageData.head(1)))

    # Get Group Data by Day
    DataByDay = DailyData.Flag_Time_By_Day.drop_duplicates()
    TotalDayCollectData = DataByDay.shape[0]
    print('\n- Total day collect {first} data:\n {second}'.format(first=DataName,second=TotalDayCollectData))
    
    # Get Success Data by Day
    Output= pd.DataFrame()
    Output['Day'] = DataByDay.copy()
    Success_Rate=[]
    for item in Output['Day']:
        TempData = ActiveMeterListWithDailyData[ActiveMeterListWithDailyData.Flag_Time_By_Day==item]
        Success_Rate.append(round(TempData.shape[0]*100/ActiveMeterList.shape[0],2))
        
    Output['Success_Rate'] = Success_Rate
    print('\n- Top 5 Success rate of collect {first} data:\n {second}'.format(first=DataName,second=Output.head(5)))
    
    # Show Chart of Success_Rate by Day
    # create data
    Output = Output.sort_values(by=['Day'])
    x=Output['Day'].head(5)
    y=Output['Success_Rate'].head(5)

    # Change the style of plot
    plt.figure(figsize=(18, 9))
    plt.xticks(rotation = 90)
    plt.style.use('seaborn-darkgrid')
    
    # Make the same graph
    plt.fill_between( x, y, color="skyblue", alpha=0.3)
    plt.plot(x, y, color="skyblue")
    
    # Add title and axis names
    plt.title('Success rate of collect {first} data last 5 days'.format(first=DataName))
    plt.xlabel('Day')
    plt.ylabel('Success rate (%)')
    
    # Show graph
    plt.show()

    
    return ActiveMeterListWithDailyData, Output


## DAILY VOLTAGE DATA

Success Rate

In [ ]:
# Success Rate (how many meter have this data in day per total active meter)
ActiveMeterListWithVoltageData, SuccessRateVoltage = getDataRate(ActiveMeterList,DailyVoltage,'Voltage')


Check Unomal Data

In [ ]:
# Voltage at Viet Nam normal from 176 V to 264 V. Check unormal data
max = 264
min = 176
UnormalVoltageData = ActiveMeterListWithVoltageData[((ActiveMeterListWithVoltageData.Voltage_in_phase_L1>max)|
                                                    (ActiveMeterListWithVoltageData.Voltage_in_phase_L1<min))|
                                                     ((ActiveMeterListWithVoltageData.Voltage_in_phase_L2>max)|
                                                    (ActiveMeterListWithVoltageData.Voltage_in_phase_L2<min))|
                                                     ((ActiveMeterListWithVoltageData.Voltage_in_phase_L3>max)|
                                                    (ActiveMeterListWithVoltageData.Voltage_in_phase_L3<min))]

TotalUnormalVoltageData = UnormalVoltageData.shape[0]
print('\n- Unormal Voltage Data :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalVoltageData,second=UnormalVoltageData))

# Three Phase meter normal must have voltage value of all Phase. Check unormal data. Just check with meter can collect data

VoltageDataThreePhaseMeter = ActiveMeterListWithVoltageData[ActiveMeterListWithVoltageData.Meter_Model=='MM31/38']
if VoltageDataThreePhaseMeter.shape[0]>0:
    VoltageDataThreePhaseMeter['CheckPhase1'] = VoltageDataThreePhaseMeter['Voltage_in_phase_L1'].isnull()
    VoltageDataThreePhaseMeter['CheckPhase2'] = VoltageDataThreePhaseMeter['Voltage_in_phase_L2'].isnull()
    VoltageDataThreePhaseMeter['CheckPhase3'] = VoltageDataThreePhaseMeter['Voltage_in_phase_L3'].isnull()

    UnormalVoltageDataThreePhaseMeter = VoltageDataThreePhaseMeter[((VoltageDataThreePhaseMeter.CheckPhase1==True)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase2==True)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase3==True))&
                                                                        ((VoltageDataThreePhaseMeter.CheckPhase1==False)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase2==False)|
                                                                        (VoltageDataThreePhaseMeter.CheckPhase3==False))]

    TotalUnormalVoltageDataThreePhaseMeter = UnormalVoltageDataThreePhaseMeter.shape[0]
    print('\n- Unormal Voltage Data Three Phase Meter :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalVoltageDataThreePhaseMeter,second=UnormalVoltageDataThreePhaseMeter))
else:
    print('\n- Do not have Current Data of Three Phase Meter')  

## DAILY CURRENT DATA

Success Rate

In [ ]:
# Success Rate (how many meter have this data in day per total active meter)
ActiveMeterListWithCurrentData, SuccessRateCurrent = getDataRate(ActiveMeterList,DailyCurrent,'Current')

Check Unomal Data

In [ ]:
# Current range of single phase meter is 0 - 80A (Imax = 80A), Three phase meter is 0 - 100A (Imax = 100A). Meter can run normal if Current value < 120% Imax. Check Meter have unomal Current
ImaxSinglePhaseMeter = 80
ImaxThreePhaseMeter = 100

# Check Current range of single phase meter
CurrentDataSinglePhaseMeter = ActiveMeterListWithCurrentData[ActiveMeterListWithCurrentData.Meter_Model=='MM11-V1']

if CurrentDataSinglePhaseMeter.shape[0]>0:
    UnormalCurrentDataSinglePhaseMeter = CurrentDataSinglePhaseMeter[CurrentDataSinglePhaseMeter.Current_in_phase_L1>ImaxSinglePhaseMeter]

    TotalUnormalCurrentDataSinglePhaseMeter = UnormalCurrentDataSinglePhaseMeter.shape[0]
    print('\n- Unormal Current Data Single Phase Meter :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalCurrentDataSinglePhaseMeter,second=UnormalCurrentDataSinglePhaseMeter))
else:
    print('\n- Do not have Current Data of Single Phase Meter')    
# Check Current range of single phase meter

CurrentDataThreePhaseMeter = ActiveMeterListWithCurrentData[ActiveMeterListWithCurrentData.Meter_Model=='MM31/38']
if CurrentDataThreePhaseMeter.shape[0]>0:
    UnormalCurrentDataThreePhaseMeter = CurrentDataThreePhaseMeter[(CurrentDataThreePhaseMeter.Current_in_phase_L1>ImaxThreePhaseMeter)|
                                                                (CurrentDataThreePhaseMeter.Current_in_phase_L2>ImaxThreePhaseMeter)|
                                                                (CurrentDataThreePhaseMeter.Current_in_phase_L3>ImaxThreePhaseMeter)]

    TotalUnormalCurrentDataThreePhaseMeter = UnormalCurrentDataThreePhaseMeter.shape[0]
    print('\n- Unormal Current Data Three Phase Meter :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalCurrentDataThreePhaseMeter,second=UnormalCurrentDataThreePhaseMeter))
else:
    print('\n- Do not have Current Data of Three Phase Meter')    

## DAILY FORWARD ENERGY DATA

Success Rate

In [ ]:
# Success Rate (how many meter have this data in day per total active meter)
df, SuccessRateForwardEnergyData = getDataRate(ActiveMeterList,DailyForwardEnergy,'Forward Energy')

Check Unomal Data

In [ ]:
# Normal Data of Enegry is the value >=0. Check that
UnormalEnegryDataCheckValue = df[(df.Active_energy_total < 0) |
                                 (df.Active_energy_tariff1 < 0) | 
                                 (df.Active_energy_tariff2 < 0) | 
                                 (df.Active_energy_tariff3 < 0) | 
                                 (df.Active_energy_tariff4 < 0) |
                                 (df.Reactive_energy_total < 0) |
                                 (df.Reactive_energy_tariff1 < 0) | 
                                 (df.Reactive_energy_tariff2 < 0) | 
                                 (df.Reactive_energy_tariff3 < 0) | 
                                 (df.Reactive_energy_tariff4 < 0)]

TotalUnormalEnegryDataCheckValue = UnormalEnegryDataCheckValue.shape[0]
print('\n- Unormal Enegery (Enegry value <0) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckValue,second=UnormalEnegryDataCheckValue))


# With multi tariff meter TotalEnegry = (99% - 101%) of (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4). Check that
minvalue = 0.99
maxvalue = 1.01
UnormalEnegryDataCheckTotalAndTariffValue = df[((df.Active_energy_total < (df.Active_energy_tariff1 + df.Active_energy_tariff2 + df.Active_energy_tariff3 + df.Active_energy_tariff4)*minvalue)|
                                               (df.Active_energy_total > (df.Active_energy_tariff1 + df.Active_energy_tariff2 + df.Active_energy_tariff3 + df.Active_energy_tariff4)*maxvalue))|
                                               ((df.Reactive_energy_total < (df.Reactive_energy_tariff1 + df.Reactive_energy_tariff2 + df.Reactive_energy_tariff3 + df.Reactive_energy_tariff4)*minvalue)|
                                               (df.Reactive_energy_total > (df.Reactive_energy_tariff1 + df.Reactive_energy_tariff2 + df.Reactive_energy_tariff3 + df.Reactive_energy_tariff4)*maxvalue))]

TotalUnormalEnegryDataCheckTotalAndTariffValue = UnormalEnegryDataCheckTotalAndTariffValue.shape[0]
print('\n- With multi tariff meter, Unormal Enegery (TotalEnegry not same with (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4)) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckTotalAndTariffValue,second=UnormalEnegryDataCheckTotalAndTariffValue))

## DAILY FORWARD ENERGY DATA

Success Rate

In [ ]:
# Success Rate (how many meter have this data in day per total active meter)
df, SuccessRateReverseEnergyData = getDataRate(ActiveMeterList,DailyReverseEnergy,'Reverse Energy')

Check Unomal Data

In [ ]:
# Normal Data of Enegry is the value >=0. Check that
UnormalEnegryDataCheckValue = df[(df.Active_energy_total < 0) |
                                 (df.Active_energy_tariff1 < 0) | 
                                 (df.Active_energy_tariff2 < 0) | 
                                 (df.Active_energy_tariff3 < 0) | 
                                 (df.Active_energy_tariff4 < 0) |
                                 (df.Reactive_energy_total < 0) |
                                 (df.Reactive_energy_tariff1 < 0) | 
                                 (df.Reactive_energy_tariff2 < 0) | 
                                 (df.Reactive_energy_tariff3 < 0) | 
                                 (df.Reactive_energy_tariff4 < 0)]

TotalUnormalEnegryDataCheckValue = UnormalEnegryDataCheckValue.shape[0]
print('\n- Unormal Enegery (Enegry value <0) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckValue,second=UnormalEnegryDataCheckValue))


# With multi tariff meter TotalEnegry = (99% - 101%) of (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4). Check that
minvalue = 0.99
maxvalue = 1.01
UnormalEnegryDataCheckTotalAndTariffValue = df[((df.Active_energy_total < (df.Active_energy_tariff1 + df.Active_energy_tariff2 + df.Active_energy_tariff3 + df.Active_energy_tariff4)*minvalue)|
                                               (df.Active_energy_total > (df.Active_energy_tariff1 + df.Active_energy_tariff2 + df.Active_energy_tariff3 + df.Active_energy_tariff4)*maxvalue))|
                                               ((df.Reactive_energy_total < (df.Reactive_energy_tariff1 + df.Reactive_energy_tariff2 + df.Reactive_energy_tariff3 + df.Reactive_energy_tariff4)*minvalue)|
                                               (df.Reactive_energy_total > (df.Reactive_energy_tariff1 + df.Reactive_energy_tariff2 + df.Reactive_energy_tariff3 + df.Reactive_energy_tariff4)*maxvalue))]

TotalUnormalEnegryDataCheckTotalAndTariffValue = UnormalEnegryDataCheckTotalAndTariffValue.shape[0]
print('\n- With multi tariff meter, Unormal Enegery (TotalEnegry not same with (EnegryTariff1 + EnegryTariff2 + EnegryTariff3 + EnegryTariff4)) :\n+ Total: {first} \n+ Detail:\n{second}'.format(first=TotalUnormalEnegryDataCheckTotalAndTariffValue,second=UnormalEnegryDataCheckTotalAndTariffValue))
